In [24]:
import pandas as pd
df = pd.read_csv("areas-export.csv")
# print(df.dtypes)
df["level"] = df["level"].round().astype('Int64')
df["parent_id"] = df["parent_id"].round().astype('Int64')
# print(df.dtypes)
# print(df.head())
df = df.rename(columns={"title": "name"})
df.to_json("areas.json", orient='records')

In [25]:
df.dtypes

id             int64
name          object
parent_url    object
url           object
routes          bool
level          Int64
parent_id      Int64
dtype: object

In [20]:
df = pd.read_csv("routes-export.csv")
print(df.dtypes)
df
df["pitches"] = df["pitches"].round().astype('Int64')
# df["parent_id"] = df["parent_id"].round().astype('Int64')
# print(df.dtypes)
# print(df.head())

df = df.rename(columns={"starVotes": "star_votes", "imgSqSmall": "img_sq_small", "imgSmall": "img_small", "imgSmallMed": "img_small_med", "imgMedium": "img_medium", "locationId": "location_id"})

df.to_json("routes.json", orient='records')

id               int64
name            object
type            object
rating          object
stars          float64
starVotes        int64
pitches        float64
location        object
url             object
imgSqSmall      object
imgSmall        object
imgSmallMed     object
imgMedium       object
longitude      float64
latitude       float64
locationId       int64
dtype: object


In [22]:
# Read in the file
with open('routes.json', 'r') as file:
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('\/', '/')

# Write the file out again
with open('routes.json', 'w') as file:
  file.write(filedata)

In [29]:
userIds = [
    200249441,
    200415321,
    200227434,
    200386524,
    110534118,
    200314827,
    200018075,
    200370327,
    200155809,
    200527701
]

In [34]:
import requests
mpkey = "200157044-e39bcc082e1d854263a47239d609d945"
def getMPUser(email=None, userId=None):
    baseurl = "https://www.mountainproject.com/data/get-user?"
    if email is None and userId is None:
        raise Exception("No valid args")
    if userId:
        params = {
            "userId": userId,
            "key": mpkey
        }
    else:
        params = {
            "email": email,
            "key": mpkey
        }
    R = requests.get(baseurl, params=params)
    return(R.json())

def getTicks(email=None, userId=None, startPos=0):
    baseurl = "https://www.mountainproject.com/data/get-ticks?"
    if email is None and userId is None:
        raise Exception("No valid args")
    if userId:
        params = {
            "userId": userId,
            "key": mpkey,
            "startPos": startPos
        }
    else:
        params = {
            "email": email,
            "key": mpkey,
            "startPos": startPos
        }
    R = requests.get(baseurl, params=params)
    return(R.json())
    # return(json.loads(R.json()))

def getAllTicks(email=None,userId=None):
    if email is None and userId is None:
        raise Exception("No valid args")
    pagenum = 0
    all = []
    rerun = False
    current = getTicks(email=email,userId=userId,startPos=pagenum)
    all.extend(current["ticks"])
    if(len(current["ticks"]) == 200):
        rerun = True
    while(rerun):
        pagenum += 200
        current = getTicks(email=email,userId=userId,startPos=pagenum)
        all.extend(current["ticks"])
        if(len(current["ticks"]) == 200):
            rerun = True
        else:
            rerun = False
    # return jsonify(all)
    return all

def divide_chunks(l, n):
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]

def getRoutes(ids):
# def getRoutes(ids, email=None, userId=None):
    # get user API key from database
    # if email is None and userId is None:
    #     raise Exception("No valid email or ID")

    baseurl = "https://www.mountainproject.com/data/get-routes?"
    routeIds = [str(id) for id in ids]
    routeIds_str = ",".join(routeIds)
    print(routeIds_str)
    params = {
        "key": mpkey,
        "routeIds": routeIds_str
    }
    R = requests.get(baseurl, params=params)
    return(R.json())

def getAllRoutes(ids):
# def getAllRoutes(ids, email=None, userId=None):
    if(len(ids) <= 200):
        return getRoutes(ids)
    else:
        # break into 200 chunks, go get those
        print("chunking")
        allRoutes = []
        routes = list(divide_chunks(ids, 200))
        for block in routes:
            # allRoutes.extend(getRoutes(block, email=email, userId=userId)['routes'])
            allRoutes.extend(getRoutes(block)['routes'])
        return allRoutes

def searchMPUsers(name, offset=0, sort="Default"):
    # Can also sort by Name
    # 50 at a time
    baseurl = 'https://www.mountainproject.com/ajax/public/search/results/category'
    params = {
        "q":name,
        "c":"Users",
        "o":offset,
        "s":sort
    }
    R = requests.get(baseurl, params=params)
    # print(R.url)
    users = R.json()["results"]["Users"]
    return users

In [41]:
profiles = []
all_ticks = []
for userId in userIds:
    user = getMPUser(userId=userId)
    ticks = getAllTicks(userId=userId)
    for tick in ticks:
        tick["mp_user_id"] = userId
        tick["mp_name"] = user["name"]
    profiles.append(user)
    all_ticks.extend(ticks)

In [37]:
all_ticks

[{'routeId': 116170613,
  'date': '2020-11-14',
  'pitches': 1,
  'notes': 'Fun, could take 4 #3s',
  'style': 'Follow',
  'leadStyle': '',
  'tickId': 119919175,
  'userStars': -1,
  'userRating': '',
  'mp_user_id': 200249441},
 {'routeId': 111311121,
  'date': '2020-11-14',
  'pitches': 1,
  'notes': 'Took after 3rd perma - go to small hueco to left of draw to clip next time. Then did the next few draws clean, took at the hard clip with the two crimps right above it',
  'style': 'Lead',
  'leadStyle': 'Fell/Hung',
  'tickId': 119919174,
  'userStars': -1,
  'userRating': '',
  'mp_user_id': 200249441},
 {'routeId': 111311121,
  'date': '2020-11-14',
  'pitches': 1,
  'notes': 'Fell after 4th perms, figuring out the weird left hand. Took at the next few draws as well. Lots of knee bars for left leg',
  'style': 'Lead',
  'leadStyle': 'Fell/Hung',
  'tickId': 119919173,
  'userStars': -1,
  'userRating': '',
  'mp_user_id': 200249441},
 {'routeId': 110356289,
  'date': '2020-11-14',
 

In [40]:
df = pd.DataFrame(data=profiles)
df.to_json("profile.json", orient='records')

In [42]:
df = pd.DataFrame(data=all_ticks)
df = df.rename(columns={"routeId":"route_id", "leadStyle": "lead_style", "tickId": "id", "userStars":"user_stars", "userRating": "user_rating"})
df.to_json("tick.json", orient='records')

In [50]:
routeIds = set()
for tick in all_ticks:
    route = tick['routeId']
    routeIds.add(route)
print(len(routeIds))

1207


In [57]:
areas = pd.read_csv("areas-export.csv")
areas["level"] = areas["level"].round().astype('Int64')
areas["parent_id"] = areas["parent_id"].round().astype('Int64')
# print(df.dtypes)
# print(df.head())
areas = areas.rename(columns={"title": "name"})
# areas.to_json("areas.json", orient='records')

In [48]:
routes = pd.read_csv("routes-export.csv")
routes["pitches"] = routes["pitches"].round().astype('Int64')
routes = routes.rename(columns={"starVotes": "star_votes", "imgSqSmall": "img_sq_small", "imgSmall": "img_small", "imgSmallMed": "img_small_med", "imgMedium": "img_medium", "locationId": "location_id"})

# routes.to_json("routes.json", orient='records')

In [49]:
routes

,id,name,type,rating,stars,star_votes,pitches,location,url,img_sq_small,img_small,img_small_med,img_medium,longitude,latitude,location_id
0,105714687,Bridalveil Falls Right Side,"Trad, Ice",WI2+,4.2,5,<NA>,"['South Dakota', 'Spearfish Canyon', 'Bridalve...",https://www.mountainproject.com/route/10571468...,https://cdn2.apstatic.com/photos/climb/1066423...,https://cdn2.apstatic.com/photos/climb/1066423...,https://cdn2.apstatic.com/photos/climb/1066423...,https://cdn2.apstatic.com/photos/climb/1066423...,-103.8589,44.4578,105714675
1,105714691,Honey Bear Main Flow,Ice,WI3,3.2,6,1,"['South Dakota', 'Custer State Park', 'Custer ...",https://www.mountainproject.com/route/10571469...,https://cdn2.apstatic.com/photos/climb/1074535...,https://cdn2.apstatic.com/photos/climb/1074535...,https://cdn2.apstatic.com/photos/climb/1074535...,https://cdn2.apstatic.com/photos/climb/1074535...,-103.5413,43.8697,105714678
2,105714695,Terminator Pillar,"Trad, Ice",WI4,4.7,6,<NA>,"['South Dakota', 'Spearfish Canyon', 'Terminat...",https://www.mountainproject.com/route/10571469...,https://cdn2.apstatic.com/photos/climb/1030007...,https://cdn2.apstatic.com/photos/climb/1030007...,https://cdn2.apstatic.com/photos/climb/1030007...,https://cdn2.apstatic.com/photos/climb/1030007...,-103.8589,44.4578,105714681
3,105714699,11th Hour Center,"Trad, TR, Ice",WI3,3.5,12,<NA>,"['South Dakota', 'Spearfish Canyon', '11th Hou...",https://www.mountainproject.com/route/10571469...,https://cdn2.apstatic.com/photos/climb/1020014...,https://cdn2.apstatic.com/photos/climb/1020014...,https://cdn2.apstatic.com/photos/climb/1020014...,https://cdn2.apstatic.com/photos/climb/1020014...,-103.8589,44.4578,105714684
4,105714703,Gorillas in the Schist,"Trad, Mixed, Ice",5.9 WI3 M5 X,2.5,2,<NA>,"['South Dakota', 'Spearfish Canyon', 'Bridalve...",https://www.mountainproject.com/route/10571470...,NaN,NaN,NaN,NaN,-103.8589,44.4578,105714675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228282,119562534,Long Forgotten,"Trad, Alpine",5.10a,5.0,1,4,"['International', 'North America', 'Canada', '...",https://www.mountainproject.com/route/11956253...,https://cdn2.apstatic.com/photos/climb/1195625...,https://cdn2.apstatic.com/photos/climb/1195625...,https://cdn2.apstatic.com/photos/climb/1195625...,https://cdn2.apstatic.com/photos/climb/1195625...,-118.0589,52.6822,107279312
228283,119562555,Giant Steps,Sport,5.11b,4.0,1,<NA>,"['Washington', 'Central-East Cascades, Wenatch...",https://www.mountainproject.com/route/11956255...,NaN,NaN,NaN,NaN,-120.7052,47.5492,106011313
228284,119562746,Wrecking My Zen,"Trad, Aid",5.9 A0,3.0,1,3,"['New York', 'Adirondacks', 'High Peaks Region...",https://www.mountainproject.com/route/11956274...,https://cdn2.apstatic.com/photos/climb/1195632...,https://cdn2.apstatic.com/photos/climb/1195632...,https://cdn2.apstatic.com/photos/climb/1195632...,https://cdn2.apstatic.com/photos/climb/1195632...,-73.9066,44.1133,109235496
228285,119562754,Kingspitz - Nordostwand - via Steuri,Trad,5.10a,5.0,1,15,"['International', 'Europe', 'Switzerland', 'En...",https://www.mountainproject.com/route/11956275...,https://cdn2.apstatic.com/photos/climb/1195627...,https://cdn2.apstatic.com/photos/climb/1195627...,https://cdn2.apstatic.com/photos/climb/1195627...,https://cdn2.apstatic.com/photos/climb/1195627...,8.1807,46.6827,107417228


In [52]:
small_routes = routes[routes.id.isin(routeIds)]
small_routes.to_json("small_routes.json", orient='records')

In [55]:
area_ids = small_routes['location_id'].tolist()

In [58]:
small_areas = areas[areas.id.isin(area_ids)]
small_areas.to_json("small_areas.json", orient="records")